In [ ]:
# load  dataset
from datasets import load_dataset
dataset = load_dataset("blanchon/snac_llm_parler_tts") # needs datasets ~2.19.1

sample = dataset["train"][0]
print(f'{int(sample["begin_time"]/60)}:{int(sample["begin_time"]%60)}min from:', sample["original_path"])

In [ ]:
# load snac model
from snac import SNAC

model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz").eval()
model = model.cuda()

In [ ]:
# restore sample from dataset

from collections import deque
from IPython.display import Audio
import torch
import soundfile as sf

AUDIO_FILE_PATH = 'data/reconstructed_audio.wav'

def reconstruct_codec(flattened):

    flattened = deque(flattened) # makes it efficient
    lists = [[],[],[]]

    while flattened:
        lists[0] += [flattened.popleft()]
        lists[1] += [flattened.popleft()]
        lists[2] += [flattened.popleft()]
        lists[2] += [flattened.popleft()]
        lists[1] += [flattened.popleft()]
        lists[2] += [flattened.popleft()]
        lists[2] += [flattened.popleft()]

    return [torch.tensor(l).unsqueeze(0).to("cuda") for l in lists]

def flat_codec(codec):

    flattened = []
    for i in range(len(codec[0][0])):
        flattened.append(codec[0][0][i])
        flattened.append(codec[1][0][2*i])
        flattened.append(codec[2][0][4*i])

        if 4*i + 1 < len(codec[2][0]):
            flattened.append(codec[2][0][4*i + 1])

        if 2*i + 1 < len(codec[1][0]):
            flattened.append(codec[1][0][2*i + 1])
            flattened.append(codec[2][0][4*i + 2])

            if 4*i + 3 < len(codec[2][0]):
                flattened.append(codec[2][0][4*i + 3])

    return flattened



with torch.inference_mode():
    tokens = [int(t) for t in sample["snac24khz"].split(" ")]
    codes = reconstruct_codec(tokens)
    #codes = reconstruct_codec(flat_codec(codes))
    audio_hat = model.decode(codes)

sf.write(AUDIO_FILE_PATH, audio_hat.cpu().detach().numpy().squeeze(), 24000)
Audio(AUDIO_FILE_PATH)